In [ ]:
!pip install pandas==1.0.3 regex matplotlib==3.2.1 cython grpcio-tools scikit-learn pyarrow fastparquet plotly memoized-property opt-einsum==2.3.2 > /dev/null 2>&1

In [ ]:
!pip install --upgrade git+https://github.com/frreiss/text-extensions-for-pandas > /dev/null 2>&1

In [ ]:
%load util.py

# maps_us_data.ipynb

Display interactive maps of the most recent COVID-19 statistics with and without normalization by population.

Inputs:
* `data/us_counties_clean.csv`: The contents of `data/us_counties.csv` after data cleaning by `clean.ipynb`
* `data/us_counties_clean_meta.json`: Column type metadata for reading `data/us_counties_clean.csv` with `pd.read_csv()`
* [U.S. map in GeoJSON format, from Plotly](https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json)

In [ ]:
# Initialization boilerplate

# Ensure a consistent Python environment.
#import sys
#sys.path.append("..")  # Local libraries are in the directory above "notebooks"
#import env
#env.maybe_install_libs()

import os
import json
import pandas as pd
import numpy as np
from urllib.request import urlopen

from typing import *

import text_extensions_for_pandas as tp

# Local file of utility functions
import util

_DATA_DIR = os.getenv('COVID_DATA_DIR', '../data')

In [ ]:
# Read in the CSV file and apply the saved type information
csv_file = os.path.join(_DATA_DIR, "us_counties_clean.csv")
meta_file = os.path.join(_DATA_DIR, "us_counties_clean_meta.json")

# Read column type metadata
with open(meta_file) as f:
    cases_meta = json.load(f)

# Pandas does not currently support parsing datetime64 from CSV files.
# As a workaround, read the "Date" column as objects and manually 
# convert after.
cases_meta["Date"] = "object"

cases_vertical = pd.read_csv(csv_file, dtype=cases_meta, parse_dates=["Date"])
cases_vertical

In [ ]:
# Slice off the last element of each time series.
cases = cases_vertical[cases_vertical["Date"] == cases_vertical["Date"].max()].set_index("FIPS")
cases

In [ ]:
# Normalize the Confirmed and Deaths counts by population.
cases["confirmed_per_100"] = cases["Confirmed"] / cases["Population"] * 100
cases["deaths_per_100"] = cases["Deaths"] / cases["Population"] * 100

cases

In [ ]:
# Download a U.S. map in GEOJSON format
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
# Common code to generate choropleth maps.
# NOTE: In order for this to work you need the JupyterLab extensions for Plotly:
#   > jupyter labextension install jupyterlab-plotly
# (env.sh will run the above command for you)

import plotly.express as px

def draw_map(col_name, label_str):
    # Each series may have NAs in different locations
    valid_data = cases[~cases[col_name].isna()]
    
    fig = px.choropleth(valid_data, geojson=counties, 
                        locations=["{:05d}".format(f) for f in valid_data.index],
                        color=col_name,
                        # See https://plotly.com/python/builtin-colorscales/
                        color_continuous_scale="viridis",
                        # Top of scale == 95th percentile
                        range_color=(0, valid_data[col_name].quantile(0.95)),
                        scope="usa",
                        labels={col_name: label_str},
                        hover_name=valid_data["County"],
                        title=label_str)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()
    


In [ ]:
# Draw a map of number of confirmed cases by county.
# Yellow == 95th percentile
draw_map("Confirmed", "Confirmed Cases ")

In [ ]:
# Draw a map of number of confirmed cases per 100 residents by county
# Yellow == 95th percentile
draw_map("confirmed_per_100", "Confirmed per 100")

In [ ]:
# Draw a map of number of deaths by county
# Yellow == 95th percentile
draw_map("Deaths", "Total Deaths  ")

In [ ]:
# Draw a map of number of deaths per 100 residents by county
# Yellow == 95th percentile
draw_map("deaths_per_100", "Deaths per 100")